Input

In [2]:
from pathlib import Path 
sample = Path('sample').read_text()
input = Path('input').read_text()

Parsing

In [3]:
import re
rx = re.compile(r'^Valve (\w\w) has flow rate=(\d+); tunnels? leads? to valves? (.*)$')

def _parse(input):
  for line in input.splitlines():
    valve, flow, valves = rx.match(line).groups()
    yield valve, (int(flow), valves.split(', '))

def parse(input):
  return dict(_parse(input))

parse(sample)

{'AA': (0, ['DD', 'II', 'BB']),
 'BB': (13, ['CC', 'AA']),
 'CC': (2, ['DD', 'BB']),
 'DD': (20, ['CC', 'AA', 'EE']),
 'EE': (3, ['FF', 'DD']),
 'FF': (0, ['EE', 'GG']),
 'GG': (0, ['FF', 'HH']),
 'HH': (22, ['GG']),
 'II': (0, ['AA', 'JJ']),
 'JJ': (21, ['II'])}

Solution

In [29]:
# def solve(input):
#   valves = parse(input)
#   mem = dict()

#   def f(valve, total_pressure, time_left, open_valves):
#     if time_left == 0:
#       return 0
#     elif all(valves[valve][0] == 0 or valve in open_valves for valve in valves):
#       return total_pressure * time_left

#     key = (valve, total_pressure, time_left, tuple(open_valves))

#     if key not in mem:
#       result = []
#       # 1. do nothing
#       result.append(f(valve, total_pressure, time_left - 1, open_valves))
#       # 2. open valve
#       if valves[valve][0] > 0 and valve not in open_valves:
#         result.append(f(valve, valves[valve][0] + total_pressure, time_left - 1, sorted(open_valves + [valve])))
#       # 3. move to next valve
#       for next_valve in valves[valve][1]:
#         result.append(f(next_valve, total_pressure, time_left - 1, open_valves))
      
#       mem[key] = max(result) + total_pressure

#     return mem[key]
  
#   return f('AA', 0, 30, [])

def seconds(it):
  return (e for (_, e) in it)


def solve(input):
  valves = parse(input)
  mem = dict()

  mem2 = dict()
  def pressure(open_valves):
    key = open_valves
    if key not in mem2:
      mem2[key] = sum(valves[valve][0] * time_left for (time_left, valve) in open_valves)
    return mem2[key]

  def f(valve, time_left, open_valves):
    if time_left == 0:
      return open_valves
    elif all(valves[valve][0] == 0 or valve in seconds(open_valves) for valve in valves):
      return open_valves
    
    key = (valve, time_left, tuple(seconds(open_valves)))

    if key not in mem:
      result = []
      # 1. do nothing
      result.append(f(valve, time_left - 1, open_valves))
      # 2. open valve
      if valves[valve][0] > 0 and valve not in seconds(open_valves):
        result.append(f(valve, time_left - 1, tuple(sorted(open_valves + ((time_left, valve),)))))
      # 3. move to next valve
      for next_valve in valves[valve][1]:
        result.append(f(next_valve, time_left - 1, open_valves))
      mem[key] = max(result, key=lambda open_valves: pressure(open_valves))

    return mem[key]
  
  return f('AA', 30, ())

assert solve(sample) == 1651
# solve(input)

KeyboardInterrupt: 

In [22]:
from itertools import combinations

def solve(input):
  valves = parse(input)
  mem = dict()

  def f(valve, total_pressure, time_left, open_valves):
    key = (valve, total_pressure, time_left, tuple(open_valves))

    if key not in mem:
      if time_left == 0:
        mem[key] = 0, tuple(open_valves)
      elif all(valves[valve][0] == 0 or valve in open_valves for valve in valves):
        mem[key] = total_pressure * time_left, tuple(open_valves)
      else:
        result = []
        # 1. do nothing
        result.append(f(valve, total_pressure, time_left - 1, open_valves))
        # 2. open valve
        if valves[valve][0] > 0 and valve not in open_valves:
          result.append(f(valve, valves[valve][0] + total_pressure, time_left - 1, sorted(open_valves + [valve])))
        # 3. move to next valve
        for next_valve in valves[valve][1]:
          result.append(f(next_valve, total_pressure, time_left - 1, open_valves))
        
        pressure, open_valves = max(result, key=lambda x: x[0])
        mem[key] = pressure + total_pressure, open_valves

    return mem[key]
  
  # fill mem
  f('AA', 0, 26, [])

  # filter mem
  mem = [(set(open_valves), value) for (_, _, time_left, _), (value, open_valves) in mem.items() if time_left == 0]
  print(mem)

  # interesting_valves = set(valve for valve in valves if valves[valve][0] > 0)

  # for length in range(0, len(interesting_valves) + 1):
  #   for mine in combinations(interesting_valves, length):
  #     mine = set(mine)
  #     elephant = interesting_valves - mine
      
  #     for valves, 



solve(sample)
# assert solve(sample) == 1651
# solve(input)

[(set(), 0), (set(), 0), (set(), 0), (set(), 0), ({'DD'}, 0), (set(), 0), (set(), 0), (set(), 0), ({'BB'}, 0), ({'DD'}, 0), ({'DD'}, 0), ({'DD'}, 0), ({'CC'}, 0), ({'EE'}, 0), (set(), 0), ({'JJ'}, 0), ({'BB'}, 0), ({'BB'}, 0), ({'DD', 'CC'}, 0), ({'DD'}, 0), ({'DD'}, 0), ({'EE', 'DD'}, 0), ({'DD'}, 0), ({'CC'}, 0), ({'CC'}, 0), ({'EE'}, 0), ({'EE'}, 0), (set(), 0), ({'JJ'}, 0), ({'BB', 'CC'}, 0), ({'BB'}, 0), ({'BB'}, 0), ({'DD', 'CC'}, 0), ({'DD', 'CC'}, 0), ({'BB', 'DD'}, 0), ({'DD'}, 0), ({'EE', 'DD'}, 0), ({'EE', 'DD'}, 0), ({'DD'}, 0), ({'CC'}, 0), ({'CC'}, 0), ({'BB', 'CC'}, 0), ({'EE'}, 0), ({'EE'}, 0), ({'EE'}, 0), (set(), 0), ({'JJ'}, 0), ({'BB', 'CC'}, 0), ({'BB', 'DD'}, 0), ({'BB'}, 0), ({'BB'}, 0), ({'DD', 'CC'}, 0), ({'DD', 'CC'}, 0), ({'DD', 'BB', 'CC'}, 0), ({'BB', 'DD'}, 0), ({'BB', 'DD'}, 0), ({'JJ', 'DD'}, 0), ({'EE', 'DD'}, 0), ({'EE', 'DD'}, 0), ({'EE', 'DD'}, 0), ({'DD'}, 0), ({'CC'}, 0), ({'EE', 'CC'}, 0), ({'CC'}, 0), ({'BB', 'CC'}, 0), ({'EE'}, 0), ({'EE', 'CC'}

In [49]:
def solve(input):
  tunnels = parse(input)
  mem = dict()

  def moves(valve, open_valves):
    yield 'noop'
    if tunnels[valve][0] > 0 and valve not in open_valves:
      yield 'open'
    yield from tunnels[valve][1]

  def f(valves, total_pressure, time_left, open_valves):
    if time_left == 0:
      return 0
    elif all(tunnels[valve][0] == 0 or valve in open_valves for valve in tunnels):
      return total_pressure * time_left

    key = (valves, total_pressure, time_left, tuple(open_valves))

    if key not in mem:
      result = []

      for a in moves(valves[0], open_valves):
        for b in moves(valves[1], open_valves):
          new_valves = valves
          new_total_pressure = total_pressure
          new_open_valves = open_valves

          if a == 'open':
            if valves[0] in new_open_valves:
              continue
            new_total_pressure += tunnels[valves[0]][0]
            new_open_valves = new_open_valves + [valves[0]]
          elif a != 'noop':
            new_valves = a, new_valves[1]
          
          if b == 'open':
            if valves[1] in new_open_valves:
              continue
            new_total_pressure += tunnels[valves[1]][0]
            new_open_valves = new_open_valves + [valves[1]]
          elif b != 'noop':
            new_valves = new_valves[0], b
          
          result.append(f(tuple(sorted(new_valves)), new_total_pressure, time_left - 1, sorted(new_open_valves)))

      mem[key] = max(result) + total_pressure

    return mem[key]
  
  return f(('AA', 'AA'), 0, 26, [])

# solve(sample)
# assert solve(sample) == 1707
solve(input)

KeyboardInterrupt: 